In [3]:
import formulas
import utilities
import topics
import pandas as pd
import os
import re
from IPython import embed

if __name__ == '__main__':

    # Get inputs
    website_creator_dir = os.getcwd()
    docs_dir = utilities.get_docs_path(website_creator_dir)
    formula_file_path = website_creator_dir + os.path.sep + 'formulas.csv'
    order_file_path = website_creator_dir + os.path.sep + 'sort_orders.csv'
    syllabus_file_path = (website_creator_dir + os.path.sep + 
                          'syllabus_topics.csv')
    sort_orders_df = pd.read_csv(order_file_path)
    syllabus_df = pd.read_csv(syllabus_file_path)
    formulas_input_df = pd.read_csv(formula_file_path)
    formula_sheet_items = (
        formulas.get_formulas_on_formula_sheet(formulas_input_df))
    formula_proof_required_items = (
        formulas.get_formulas_where_proofs_required(formulas_input_df))
    
    utilities.delete_directory_if_it_exists(docs_dir)
    
    # Create formula-related markdown documents
    formulas_df = formulas.get_formulas_df(formulas_input_df, syllabus_df,
                                           sort_orders_df)
    formulas.create_formulas_content(
        formulas_df, formula_sheet_items, formula_proof_required_items, 
        sort_orders_df, docs_dir)
    formulas.create_calculus_summary_files(
        formulas_df, formula_sheet_items, formula_proof_required_items, 
        sort_orders_df, docs_dir)
    formulas.create_financial_summary(
        formulas_df, formula_sheet_items, formula_proof_required_items, 
        sort_orders_df, docs_dir)

    # Generate topic pages
    formulas_by_topic_df = formulas.get_formulas_by_topic_df(
        formulas_input_df, syllabus_df)


    # Create topic related markdown documents
    # topics_df = topics.get_topics_df(sort_orders_df)
    # topics_dir = website_creator_dir + os.path.sep + 'topic_summaries'
    # topics.create_topics_content(topics_df, sort_orders_df, docs_dir, 
    #                              topics_dir)

In [2]:
def create_topic_content(formulas_by_topic_df, formula_sheet_items,
                         formula_proof_required_items, sort_orders_df,
                         docs_dir): 
    """Creates markdown files containing topic summary for static 
    web page creation via Hugo
    """
    dirs_df = formulas_by_topic_df[[
        'State', 'Topic subcategory 1', 'Subject code']].drop_duplicates()
    file_paths_df = formulas_by_topic_df[[
        'State', 'Topic subcategory 1', 'Subject code', 'Syllabus subtopic'
        ]].drop_duplicates()
    utilities.create_subdirectories_from_df(base_dir = docs_dir, 
                                             subpaths_df = dirs_df)
    front_matter_index_files = {'bookCollapseSection' : True}
    utilities.create_index_files(base_dir=docs_dir, dirs_df=dirs_df, 
                       front_matter=front_matter_index_files,
                       sort_orders_df = sort_orders_df)
    utilities.create_files(base_dir = docs_dir, file_paths_df= file_paths_df, 
                           file_extension='.md', 
                           fn=generate_topic_page, 
                           sort_orders_df = sort_orders_df,
                           formulas_df = formulas_by_topic_df, 
                           formula_sheet_items = formula_sheet_items,
                           formula_proof_required_items = (
                               formula_proof_required_items),
                           cols_to_highlight = [
                               'Formula'])

def generate_topic_page(formulas_df, state, syllabus_subtopic,
                               subject_code, formula_sheet_items, 
                               formula_proof_required_items, 
                               cols_to_highlight, **kwargs):                                   
    """TBA"""    
    formulas_df = formulas_df[
        (formulas_df['State'] == state) &
        (formulas_df['Syllabus subtopic'] == syllabus_subtopic) &
        (formulas_df['Subject code'] == subject_code)]
    formulas_df = formulas_df[['Formula']]
    formula_string = formulas.generate_formula_string(formulas_df, 
                                             formula_sheet_items,
                                             formula_proof_required_items, 
                                             cols_to_highlight)
    definitions_string = formulas.generate_definition_string()
    page_string = ('# ' + syllabus_subtopic + '\n\n<br>\n\n'
                   + '## Definitions' + '\n\n'
                   + definitions_string + '\n\n<br>\n\n'
                   + '## Formulas' + '\n\n'
                   + formula_string
                  )
                   
    return(page_string)


if __name__ == '__main__':
    create_topic_content(formulas_by_topic_df, formula_sheet_items,
                             formula_proof_required_items, sort_orders_df,
                             docs_dir)

In [3]:
# Temp markdown file export of selected topic notebook

import utilities

input_file = '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/topic_summaries/financial_mathematics.ipynb'
output_file = '/home/charl/Onedrive/Documents_Charl/Computer_Technical/Programming_GitHub/AustralianSchoolMaths/WebsiteCreator/topic_summaries/temp.md'


export_string = utilities.filtered_notebook_md_string(
    input_notebook_path=input_file, include_tags=['Year 12 Adv'], 
    remove_input_tags=['hide_code'])

with open(output_file, "w") as text_file:
    text_file.write(export_string)

In [ ]:
def fn ():
    print('Hello, world!')

mylist = [1,2,4,fn]

mylist[3]()

Hello, world!
